In [4]:
# 必要なライブラリのインポート
import pandas as pd
import sqlite3
from geopy.geocoders import Nominatim
import tqdm

# 作成済のdatabaseに接続する。
conn = sqlite3.connect('fudosan.db')

# databaseをクエリで実行した結果を読み込み、dfに代入。
query = "SELECT * FROM suumo"
df = pd.read_sql(query, conn)

In [6]:
# Addressカラムの各要素に対して末尾が1〜9であれば「丁目」を結合する
df['Address'] = df['Address'].apply(lambda x: x + '丁目' if x[-1] in '１２３４５６７８９' else x)
df.head()

,Name,Address,Access1,Access2,Access3,Access1_Walk,Access2_Walk,Access3_Walk,Age,Max_Floor,Floor,Rent,Layout,Size,Image_URL
0,sunrise 浅草田原町,東京都台東区西浅草１丁目,東京メトロ銀座線/田原町駅,つくばエクスプレス/浅草駅,都営浅草線/浅草駅,1,3,8,2,10,7,17.1,2K,41.68,https://img01.suumo.com/front/gazo/fr/bukken/8...
1,東京メトロ南北線 四ツ谷駅 11階建 築2年,東京都新宿区左門町,東京メトロ南北線/四ツ谷駅,東京メトロ丸ノ内線/四谷三丁目駅,ＪＲ中央線/信濃町駅,14,3,7,2,11,4,14.4,2K,26.04,https://img01.suumo.com/front/gazo/fr/bukken/0...
2,東京メトロ南北線 四ツ谷駅 11階建 築2年,東京都新宿区左門町,東京メトロ南北線/四ツ谷駅,東京メトロ丸ノ内線/四谷三丁目駅,ＪＲ中央線/信濃町駅,14,3,7,2,11,6,14.5,2K,26.04,https://img01.suumo.com/front/gazo/fr/bukken/0...
3,パークアクシス押上レジデンス,東京都墨田区向島３丁目,東武伊勢崎線/とうきょうスカイツリー駅,都営浅草線/本所吾妻橋駅,東京メトロ半蔵門線/押上駅,6,10,10,6,7,5,16.7,2DK,42.21,https://img01.suumo.com/front/gazo/fr/bukken/7...
4,パークアクシス押上レジデンス,東京都墨田区向島３丁目,東武伊勢崎線/とうきょうスカイツリー駅,都営浅草線/本所吾妻橋駅,東京メトロ半蔵門線/押上駅,6,10,10,6,7,4,19.5,2LDK,50.34,https://img01.suumo.com/front/gazo/fr/bukken/7...


In [7]:
from geopy.geocoders import Nominatim
import pandas as pd
from tqdm import tqdm  # tqdmを直接インポート

geolocator = Nominatim(user_agent="property", timeout=10)

# dfは既に定義されたデータフレームと仮定します
# 住所データ
addresses = df['Address']  # 'Address'は住所を含む列名と仮定

# 緯度経度に変換
locations = []
for address in tqdm(addresses):  # ここを修正
    try:
        location = geolocator.geocode(address)
        if location:
            locations.append((location.latitude, location.longitude))
        else:
            locations.append((None, None))
    except Exception as e:
        locations.append((None, None))

# 緯度経度情報を新しいデータフレームに格納
df_ = pd.DataFrame(locations, columns=['lat', 'lon'])

# df_の内容を確認
print(df_)

100%|██████████| 2314/2314 [25:07<00:00,  1.54it/s]

            lat         lon
0     35.710933  139.790518
1     35.685912  139.720699
2     35.685912  139.720699
3     35.713316  139.809320
4     35.713316  139.809320
...         ...         ...
2309  35.585151  139.701679
2310  35.594009  139.697558
2311  35.577412  139.677830
2312  35.577412  139.677830
2313  35.673653  139.616628

[2314 rows x 2 columns]


In [8]:
df = pd.concat([df,df_], axis=1)
df

,Name,Address,Access1,Access2,Access3,Access1_Walk,Access2_Walk,Access3_Walk,Age,Max_Floor,Floor,Rent,Layout,Size,Image_URL,lat,lon
0,sunrise 浅草田原町,東京都台東区西浅草１丁目,東京メトロ銀座線/田原町駅,つくばエクスプレス/浅草駅,都営浅草線/浅草駅,1,3,8,2,10,7,17.1,2K,41.68,https://img01.suumo.com/front/gazo/fr/bukken/8...,35.710933,139.790518
1,東京メトロ南北線 四ツ谷駅 11階建 築2年,東京都新宿区左門町,東京メトロ南北線/四ツ谷駅,東京メトロ丸ノ内線/四谷三丁目駅,ＪＲ中央線/信濃町駅,14,3,7,2,11,4,14.4,2K,26.04,https://img01.suumo.com/front/gazo/fr/bukken/0...,35.685912,139.720699
2,東京メトロ南北線 四ツ谷駅 11階建 築2年,東京都新宿区左門町,東京メトロ南北線/四ツ谷駅,東京メトロ丸ノ内線/四谷三丁目駅,ＪＲ中央線/信濃町駅,14,3,7,2,11,6,14.5,2K,26.04,https://img01.suumo.com/front/gazo/fr/bukken/0...,35.685912,139.720699
3,パークアクシス押上レジデンス,東京都墨田区向島３丁目,東武伊勢崎線/とうきょうスカイツリー駅,都営浅草線/本所吾妻橋駅,東京メトロ半蔵門線/押上駅,6,10,10,6,7,5,16.7,2DK,42.21,https://img01.suumo.com/front/gazo/fr/bukken/7...,35.713316,139.809320
4,パークアクシス押上レジデンス,東京都墨田区向島３丁目,東武伊勢崎線/とうきょうスカイツリー駅,都営浅草線/本所吾妻橋駅,東京メトロ半蔵門線/押上駅,6,10,10,6,7,4,19.5,2LDK,50.34,https://img01.suumo.com/front/gazo/fr/bukken/7...,35.713316,139.809320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,（シニア限定）へーベルVillage西馬込,東京都大田区仲池上２丁目,都営浅草線/西馬込駅,東急池上線/久が原駅,東急池上線/池上駅,6,22,20,3,3,2,18.9,2LDK,63.36,https://img01.suumo.com/front/gazo/fr/bukken/2...,35.585151,139.701679
2310,（仮称）上池台ヘーベルメゾン,東京都大田区上池台３丁目,東急池上線/洗足池駅,東急池上線/長原駅,東急池上線/石川台駅,7,12,15,0,3,1,19.8,2LDK,56.9,https://img01.suumo.com/front/gazo/fr/bukken/8...,35.594009,139.697558
2311,グラン ノワ,東京都大田区鵜の木３丁目,東急多摩川線/鵜の木駅,東急池上線/久が原駅,東急多摩川線/沼部駅,5,12,11,0,5,2,18.2,2LDK,52.8,https://img01.suumo.com/front/gazo/fr/bukken/5...,35.577412,139.677830
2312,グラン ノワ,東京都大田区鵜の木３丁目,東急多摩川線/鵜の木駅,東急池上線/久が原駅,東急多摩川線/沼部駅,5,12,11,0,5,3,18.4,2LDK,52.8,https://img01.suumo.com/front/gazo/fr/bukken/5...,35.577412,139.677830


In [10]:
df[['Address', 'lat', 'lon']].query('lat != lat')

,Address,lat,lon
126,東京都台東区三筋１丁目,NaN,NaN
127,東京都台東区三筋１丁目,NaN,NaN
197,東京都台東区三筋１丁目,NaN,NaN
198,東京都台東区三筋１丁目,NaN,NaN
228,東京都品川区二葉２丁目,NaN,NaN
...,...,...,...
2126,東京都足立区六町３丁目,NaN,NaN
2144,東京都足立区六町１丁目,NaN,NaN
2147,東京都足立区六町２丁目,NaN,NaN
2209,東京都足立区六町１丁目,NaN,NaN


In [12]:
#必要なライブラリのインポート
import sqlite3
import pandas as pd

# SQLiteデータベースに接続（存在しない場合は新規作成）
conn = sqlite3.connect('fudosan2.db')

# データフレームをテーブルとして保存
df.to_sql('property_data', conn, if_exists='replace', index=False)

# 接続を閉じる
conn.close()